# Download SEC filing data

In [7]:
from sec_edgar_downloader import Downloader
import os, os.path
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Initialize a downloader instance. If no argument is passed
# to the constructor, the package will download filings to
# the current working directory.

currentDirectory = os.getcwd()
print(currentDirectory)
dl = Downloader(currentDirectory + '/data/download_2/')

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung


In [55]:
DIR = currentDirectory + '/data/download_2/sec-edgar-filings/'
for file in os.listdir(DIR):
    if not file.startswith('.'):
        company_dir = os.path.join(DIR, file,'10-K/')
        print('10K - ' + file + ': ' + str(len(os.listdir(company_dir))))
for file in os.listdir(DIR):
    if not file.startswith('.'):
        company_dir = os.path.join(DIR, file,'10-Q/')
        print('10Q - ' + file + ': ' + str(len(os.listdir(company_dir))))
    
#0001751788
#0001744489
#10Q - 0001618921
#10Q - 0000104169:
#10Q - 0001751788
#10Q - 0001744489:

10K - 0000354950: 6
10K - 0000018230: 6
10K - 0000051143: 7
10K - 0001618921: 7
10K - 0000086312: 6
10K - 0000004962: 6
10K - 0000773840: 6
10K - 0001108524: 6
10K - 0000012927: 6
10K - 0000104169: 6
10K - 0001403161: 7
10K - 0001751788: 3
10K - 0000732712: 7
10K - 0000731766: 6
10K - 0000021344: 6
10K - 0000063908: 7
10K - 0000080424: 6
10K - 0000320193: 6
10K - 0000886982: 6
10K - 0000318154: 6
10K - 0000050863: 6
10K - 0000310158: 6
10K - 0000066740: 6
10K - 0000858877: 6
10K - 0000200406: 6
10K - 0000320187: 6
10K - 0000019617: 6
10K - 0000093410: 7
10K - 0000789019: 7
10K - 0001744489: 3
10Q - 0000354950: 18
10Q - 0000018230: 18
10Q - 0000051143: 18
10Q - 0001618921: 19
10Q - 0000086312: 19
10Q - 0000004962: 19
10Q - 0000773840: 18
10Q - 0001108524: 18
10Q - 0000012927: 18
10Q - 0000104169: 18
10Q - 0001403161: 18
10Q - 0001751788: 9
10Q - 0000732712: 18
10Q - 0000731766: 18
10Q - 0000021344: 18
10Q - 0000063908: 18
10Q - 0000080424: 18
10Q - 0000320193: 18
10Q - 0000886982: 18
10

# Download dow30 10-K 10-Q

In [3]:
filename = currentDirectory + '/dow30_info.csv'
dow30_df = pd.read_csv(filename,sep=",")

In [4]:
#for index, row in dow30_df.iterrows():
#    print(row['Symbol'], row['CIK'])
#    print(str(row['CIK']).zfill(10))

In [45]:
CIK_list = dow30_df['CIK'].tolist()

In [90]:
#CIK_list = [1751788,1744489]

In [54]:
for x in CIK_list: 
    dl.get("10-Q",x,amount=18)
    dl.get("10-K",x,amount=6)

# Functions to parse report

In [57]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 3 if not w in stopwords.words('english')] ## change to 3 
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)



In [58]:
def parse_10k(company, filepath): 
    f = open(filepath)
    raw_10k = f.read()
    
    ## find start end range to avoid xml parsing problems ##
    import re
    # Regex to find <DOCUMENT> tags
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]
    
    doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]
    
    document = {}

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
        if doc_type == '10-K':
            #print(doc_start)
            #print(doc_end)
            doc_10k = raw_10k[doc_start:doc_end].lower()
            break
            
    doc_10k_bs = BeautifulSoup(doc_10k, 'lxml')
    doc_10k_str = str(doc_10k_bs)
    
    #i7 = "management’s discussion and analysis of financial condition and results of operations"
    i7 = "s discussion and analysis of financial condition and results of operations"
    i7a = 'quantitative and qualitative disclosures about market risk' 
    #if company == 'GS':
    #    i7 = 's discussion and analysis of financial condition and results of operations'
    #    i7a = 'quantitative and qualitative disclosure about market risk' 
    if company == 'INTC':
        i7 = 's discussion and analysis \(md&amp;a\)'
        i7a = 'font-weight:bold;">other key information'
    if company == 'JNJ':
        i7 = "s discussion and analysis of results of operations and financial condition"
    if company == 'MSFT':
        if read_file_year(filepath) == '2016':
            i7 = 's discussion and analysis of financial condition and results of operations </b>'
            i7a = 'market risk </b>'
        else: 
            i7a = 'ive disclosures about market risk'
    if company == 'IBM':
        i7 = 'item\xa07. management’s discussion and analysis of financial condition and results of operations.<'
    if company == 'MCD':
        if int(read_file_year(filepath)) >= 2020:
            i7 = "s discussion and analysis of financial condition and results of operations</"
            i7a = 'other key information'
        else:
            i7 = 'item\xa07. management’s discussion and analysis of financial condition and results of operations'
    if company == 'CVX':
        i7 = 'd;">key financial results'
        i7a = 'd;">quarterly results'
    if company == 'JPM':
        if read_file_year(filepath) == '2017':
            i7 = 'introduction </font>'
            i7a = 's report on internal control over financial reporting</'
        else:
            i7 = 'introduction </span></'
            i7a = 's report on internal control over financial reporting</span></div>'
    if company == 'V':
        if int(read_file_year(filepath)) >= 2019:
            i7 = 's discussion and analysis of financial condition and results of operations</span></div>'
            i7a = 'quantitative and qualitative disclosures about market risk</span></div>'
        else: 
            i7 = 'd;">management’s discussion and analysis of financial condition and results of operations</font></div></td>'
            i7a = 'quantitative and qualitative disclosures about market risk</font></div>'
    if company == 'WBA':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosure about market risk</'

        
    
    sec7_find = re.finditer(i7, doc_10k_str)
    sec7a_find = re.finditer(i7a, doc_10k_str)
    
    # moving keyword range to dataframe
    sec7_start_range = []
    sec7_end_range = []
    for x in sec7_find: 
        print('7: ' + str(x.start()))
        sec7_start_range.append(x.start())
        sec7_end_range.append(x.end())
        sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
    if not sec7_start_range: 
        print('no item 7')
        return sec7_start_range
    sec7a_start_range = []
    sec7a_end_range = []    
    for x in sec7a_find: 
        print('7a: ' + str(x.start()))
        sec7a_start_range.append(x.start())
        sec7a_end_range.append(x.end())
        sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})
    if not sec7a_start_range: 
        print('no item 7a')
        return sec7a_start_range
    
    #qc check
    #for index, row in sec7_df.iterrows():
    #    print('7' , str(row['start']) , str(row['end']))
    #for index, row in sec7a_df.iterrows():
    #    print('7a', str(row['start']), str(row['end']))
        
    # get possible ranges 
    possible_start_range = []
    possible_end_range = []
    for index, row7a in sec7a_df.iterrows():
        #print(row7a['start'])
        filtered = []
        for i, row7 in sec7_df.iterrows():
            #print(row7['end'])
            if row7['end'] < row7a['start']:
                filtered.append(row7['end'])
        #print(filtered)
        if not filtered:
            continue # nothing before item 7a
        max_y = max(filtered)
        possible_start_range.append(max_y)
        possible_end_range.append(row7a['start']-12)
    
        # qc 
        #for x in possible_start_range:
        #    print(x)

    # get the largest possible range
    possible_df = pd.DataFrame({'start': possible_start_range,'end': possible_end_range})
    possible_df['delta'] = possible_df['end'] - possible_df['start']
    if len(possible_df) == 0:
        print('exee')
        return sec7a_start_range
    print(possible_df.loc[possible_df['delta'].idxmax()])

    item7_df = possible_df.loc[possible_df['delta'].idxmax()]
    
    item_7_content = doc_10k_str[item7_df['start']:item7_df['end']]
    item_7_cleaned = preprocess(item_7_content)
    bagOfWordsA = item_7_cleaned.split(' ')
    
    return bagOfWordsA


In [92]:
def parse_10q(company, filepath): 
    f = open(filepath)
    raw_10k = f.read()
    
    ## find start end range to avoid xml parsing problems ##
    import re
    # Regex to find <DOCUMENT> tags
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]
    
    doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]
    
    document = {}

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
        if doc_type == '10-Q':
            #print(doc_start)
            #print(doc_end)
            doc_10k = raw_10k[doc_start:doc_end].lower()
            break
            
    doc_10k_bs = BeautifulSoup(doc_10k, 'lxml')
    doc_10k_str = str(doc_10k_bs)
    
    #i7 = "management’s discussion and analysis of financial condition and results of operations"
    i7 = "s discussion and analysis of financial condition and results of operations"
    i7a = 'quantitative and qualitative disclosures about market risk' 
    if company == 'INTC':
        i7 = 's discussion and analysis'
        i7a = 'font-weight:bold;">other key information'
    if company == 'IBM':
        i7 = 's discussion and analysis of results of operations and financial condition'
        i7a = 'controls and procedures'
    if company == 'AMGN':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'DOW':
        i7 = 's discussion and analysis of financial condition and results of operations.</span>'
        i7a = 'quantitative and qualitative disclosures about market risk</span>'
    if company == 'MMM':
        i7 ='s discussion and analysis of financial condition and results of operations.</'
    if company == 'MRK':
        i7 = 's discussion and analysis of financial condition and results of operations</span>'
        i7a = 'item\xa04. controls and procedures'
    if company == 'NKE':
        i7 = 's discussion and analysis of financial condition and results of operations</span>'
        i7a = 'quantitative and qualitative disclosures about market risk</span>'
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'UNH':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'TRV':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'MSFT':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'HON' and read_file_year(filepath) == '2020': # format changed after 2021
        i7 = 's discussion and analysis of<'
    if company == 'WBA':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosure about market risk</'
        #i7a = 'quantitative and qualitative disclosure about market risk</'
        
    sec7_find = re.finditer(i7, doc_10k_str)
    sec7a_find = re.finditer(i7a, doc_10k_str)
    
    
    # moving keyword range to dataframe
    sec7_start_range = []
    sec7_end_range = []
    for x in sec7_find: 
        print('7: ' + str(x.start()))
        sec7_start_range.append(x.start())
        sec7_end_range.append(x.end())
        sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
    if not sec7_start_range: 
        print('no item 2')
        return sec7_start_range
    sec7a_start_range = []
    sec7a_end_range = []    
    for x in sec7a_find: 
        print('7a: ' + str(x.start()))
        sec7a_start_range.append(x.start())
        sec7a_end_range.append(x.end())
        sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})
    if not sec7a_start_range:
        print('no item 3')
        return sec7a_start_range
    
    #qc check
    #for index, row in sec7_df.iterrows():
    #    print('7' , str(row['start']) , str(row['end']))
    #for index, row in sec7a_df.iterrows():
    #    print('7a', str(row['start']), str(row['end']))
        
    # get possible ranges 
    possible_start_range = []
    possible_end_range = []
    for index, row7a in sec7a_df.iterrows():
        #print(row7a['start'])
        filtered = []
        for i, row7 in sec7_df.iterrows():
            #print(row7['end'])
            if row7['end'] < row7a['start']:
                filtered.append(row7['end'])
        #print(filtered)
        if not filtered:
            continue # nothing before item 7a
        max_y = max(filtered)
        possible_start_range.append(max_y)
        possible_end_range.append(row7a['start']-12)
    
        # qc 
        #for x in possible_start_range:
        #    print(x)

    # get the largest possible range
    if not possible_start_range: # no possible start 
        print('no possible range')
        return sec7_start_range
    possible_df = pd.DataFrame({'start': possible_start_range,'end': possible_end_range})
    possible_df['delta'] = possible_df['end'] - possible_df['start']
    print(possible_df.loc[possible_df['delta'].idxmax()])

    item7_df = possible_df.loc[possible_df['delta'].idxmax()]
    
    item_7_content = doc_10k_str[item7_df['start']:item7_df['end']]
    item_7_cleaned = preprocess(item_7_content)
    bagOfWordsA = item_7_cleaned.split(' ')
    
    return bagOfWordsA


In [60]:
def read_file_date(filepath):
    # Regex to find <DOCUMENT> tags
    date_start_pattern = re.compile(r'FILED AS OF DATE:')
    date_end_pattern = re.compile(r'DATE AS OF CHANGE:')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    date_type_pattern = re.compile(r'<TYPE>[^\n]+')

    f = open(filepath, "r")

    raw_10k = f.read()
    document = {}

    date_start_is = [x.end() for x in date_start_pattern.finditer(raw_10k)]
    date_end_is = [x.start() for x in date_end_pattern.finditer(raw_10k)]
    date_types = [x[len('<TYPE>'):] for x in date_type_pattern.findall(raw_10k)]

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for date_type, date_start, date_end in zip(date_types, date_start_is, date_end_is):
        if date_type == '10-K':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
        if date_type == '10-Q':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
    return ddd

In [61]:
def read_file_year(filepath):
    # Regex to find <DOCUMENT> tags
    date_start_pattern = re.compile(r'FILED AS OF DATE:')
    date_end_pattern = re.compile(r'DATE AS OF CHANGE:')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    date_type_pattern = re.compile(r'<TYPE>[^\n]+')

    f = open(filepath, "r")

    raw_10k = f.read()
    document = {}

    date_start_is = [x.end() for x in date_start_pattern.finditer(raw_10k)]
    date_end_is = [x.start() for x in date_end_pattern.finditer(raw_10k)]
    date_types = [x[len('<TYPE>'):] for x in date_type_pattern.findall(raw_10k)]

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for date_type, date_start, date_end in zip(date_types, date_start_is, date_end_is):
        if date_type == '10-K':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
            yyy = ddd[0:4]
        if date_type == '10-Q':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
            yyy = ddd[0:4]
    return yyy

In [62]:
def read_conformed_period_date(filepath):
    # Regex to find <DOCUMENT> tags
    date_start_pattern = re.compile(r'CONFORMED PERIOD OF REPORT:')
    date_end_pattern = re.compile(r'FILED AS OF DATE:')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    date_type_pattern = re.compile(r'<TYPE>[^\n]+')

    f = open(filepath, "r")

    raw_10k = f.read()
    document = {}

    date_start_is = [x.end() for x in date_start_pattern.finditer(raw_10k)]
    date_end_is = [x.start() for x in date_end_pattern.finditer(raw_10k)]
    date_types = [x[len('<TYPE>'):] for x in date_type_pattern.findall(raw_10k)]

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for date_type, date_start, date_end in zip(date_types, date_start_is, date_end_is):
        if date_type == '10-K':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
        if date_type == '10-Q':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
    return ddd

In [63]:
read_file_year('/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001403161/10-K/0001403161-19-000050/full-submission.txt')

'2019'

## 10-K read using CIK

In [91]:
companyk = []
file_namek = []
doc_typek = []
file_datek = []
contentk = []
cikk = []
conformed_periodk = []
bag_lenk = []
#for x in dow30_df.: # company loop
sym_list = ['INTC', 'IBM', 'VZ', 'HON']
dow30_df2 = dow30_df[dow30_df['Symbol'].isin(sym_list)]
dow30_df2 = dow30_df
for index, row in dow30_df2.iterrows():
    x = row['CIK']
    i = 0
    currentDirectory = os.getcwd()
    file_dir = currentDirectory + '/data/download_2/'
    file_dir = file_dir + 'sec-edgar-filings/' + str(x).zfill(10) + '/10-K/'
    for name in os.listdir(file_dir): 
        if not name.startswith('.') and os.path.isfile(os.path.join(file_dir,name)+'/full-submission.txt'):
            
            if i >=7: # only get 2 10K files from each company
                break
            print(os.path.join(file_dir,name)+'/full-submission.txt')
            print(row['Symbol'])
            item7 = parse_10k(row['Symbol'], os.path.join(file_dir,name)+'/full-submission.txt')
            print(len(item7))
            bag_lenk.append(len(item7))
            contentk.append(item7)
            cikk.append(x)
            companyk.append(row['Symbol'])
            file_namek.append(name)
            doc_typek.append('10-K')
            file_d = read_file_date(os.path.join(file_dir,name)+'/full-submission.txt')
            file_datek.append(file_d)
            conformed_period_d = read_conformed_period_date(os.path.join(file_dir,name)+'/full-submission.txt')
            conformed_periodk.append(conformed_period_d)
            i = i+1
            #print(i)

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-K/0001558370-17-000479/full-submission.txt
MMM
7: 49604
7: 245524
7: 258597
7: 258979
7: 512355
7a: 63323
7a: 2207444
start     512429
end      2207432
delta    1695003
Name: 1, dtype: int64
10074
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-K/0001558370-19-000470/full-submission.txt
MMM
7: 49877
7: 250900
7: 265061
7: 265443
7: 489347
7a: 63587
7a: 2891788
start     489421
end      2891776
delta    2402355
Name: 1, dtype: int64
11067
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-K/0001558370-21-000737/full-submission.txt
MMM
7: 1167699
7: 1385884
7: 1396776
7: 1397159
7: 1573172
7: 1578947
7a: 1183877
7a: 2863048
start    1579021
end      2863036
delta    1284015
Name: 1, dtype: int64
10765
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_

4310
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000012927/10-K/0000012927-20-000014/full-submission.txt
BA
7: 589147
7: 829989
7a: 590510
7a: 1563126
start     830063
end      1563114
delta     733051
Name: 1, dtype: int64
7507
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000012927/10-K/0000012927-19-000010/full-submission.txt
BA
7: 578660
7: 811433
7a: 580023
7a: 1526275
start     811507
end      1526263
delta     714756
Name: 1, dtype: int64
7001
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000012927/10-K/0000012927-18-000007/full-submission.txt
BA
7: 36507
7: 268622
7a: 37870
7a: 992353
start    268696
end      992341
delta    723645
Name: 1, dtype: int64
6570
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000018230/10-K/0000018230-21-000063/full-submission.txt
CAT
7: 90

7: 858138
7: 911259
7: 1005140
7: 1011088
7: 1056757
7: 1115600
7: 1226241
7: 1516321
7: 1932998
7: 1935346
7a: 859492
7a: 2026728
start    1935420
end      2026716
delta      91296
Name: 1, dtype: int64
3376
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001751788/10-K/0001751788-20-000007/full-submission.txt
DOW
7: 914404
7: 966881
7: 1069168
7: 1159714
7: 1288698
7: 1548341
7: 2028271
7: 2030556
7a: 915375
7a: 2137359
start    2030630
end      2137347
delta     106717
Name: 1, dtype: int64
3281
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001751788/10-K/0001751788-22-000011/full-submission.txt
DOW
7: 804351
7: 857484
7: 968212
7: 974452
7: 1029076
7: 1088245
7: 1096498
7: 1113408
7: 1343163
7: 1737401
7: 1739827
7a: 805705
7a: 1836605
start    1739901
end      1836593
delta      96692
Name: 1, dtype: int64
3725
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data

7: 1033748
7a: 913754
7a: 1034422
start    1033843
end      1034410
delta        567
Name: 0, dtype: int64
21
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000051143/10-K/0001558370-20-001334/full-submission.txt
IBM
7: 1192985
7a: 1041976
7a: 1194388
start    1193080
end      1194376
delta       1296
Name: 0, dtype: int64
21
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000051143/10-K/0001047469-18-001117/full-submission.txt
IBM
no item 7
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000051143/10-K/0001047469-17-001061/full-submission.txt
IBM
no item 7
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000051143/10-K/0001047469-19-000712/full-submission.txt
IBM
no item 7
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/000005114

13256
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-K/0001564590-21-039151/full-submission.txt
MSFT
7: 360718
7: 411342
7: 412052
7: 446022
7: 853795
7: 854089
7: 854776
7a: 363637
7a: 412159
7a: 1396307
start     854850
end      1396295
delta     541445
Name: 2, dtype: int64
4718
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-K/0001193125-16-662209/full-submission.txt
MSFT
7: 288348
7a: 509353
start    288427
end      509341
delta    220914
Name: 0, dtype: int64
6538
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-K/0001564590-19-027952/full-submission.txt
MSFT
7: 64902
7: 115710
7: 116420
7: 615029
7a: 67821
7a: 116527
7a: 1355006
start     615103
end      1354994
delta     739891
Name: 2, dtype: int64
5272
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-ed

7: 325972
7: 327507
7: 399155
7: 399817
7: 496547
7: 502062
7: 517896
7: 520971
7: 527643
7: 543220
7: 573328
7: 600057
7: 758526
7: 2159202
7: 2162384
7: 2162946
7a: 26724
7a: 396921
7a: 1794167
7a: 2160216
start     758600
end      1794155
delta    1035555
Name: 1, dtype: int64
10889
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000086312/10-K/0000086312-19-000009/full-submission.txt
TRV
7: 33686
7: 82766
7: 370943
7: 371539
7: 373741
7: 378107
7: 482227
7: 482796
7: 684563
7: 690590
7: 708833
7: 712886
7: 742353
7: 750393
7: 785177
7: 821242
7: 920710
7: 2739350
7: 2743822
7: 2744576
7a: 34647
7a: 479908
7a: 2243240
7a: 2245685
7a: 2740231
start     920784
end      2245673
delta    1324889
Name: 3, dtype: int64
10326
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000086312/10-K/0001047469-17-000695/full-submission.txt
TRV
7: 70034
7: 284214
7: 285732
7: 352000
7: 448288
7: 453642

7: 39156
7: 1062148
7a: 40112
7a: 1516261
start    1062222
end      1516249
delta     454027
Name: 1, dtype: int64
5531
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000104169/10-K/0000104169-21-000033/full-submission.txt
WMT
7: 249457
7: 736887
7a: 250870
7a: 1098112
start     736961
end      1098100
delta     361139
Name: 1, dtype: int64
6001
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001618921-21-000085/full-submission.txt
WBA
7: 387115
7: 723526
7a: 388293
7a: 1328222
start     723602
end      1328210
delta     604608
Name: 1, dtype: int64
8820
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001628280-18-012472/full-submission.txt
WBA
7: 32800
7: 526262
7a: 33728
7a: 1228684
start     526338
end      1228672
delta     702334
Name: 1, dtype: int64
7717
/Users/wailunchung/Documents/GitHub/DS6120

In [19]:
# CVX, INTC, IBM, MSFT, .5 VZ
# CVX, HON download 1 more
# IBM no item 7, they put it in annual report 
# VZ no item 7 before 2021, they put it in annual report 



## 10-Q read using CIK

In [93]:
companyq = []
file_nameq = []
doc_typeq = []
file_dateq = []
contentq = []
cikq = []
conformed_periodq = []
bag_lenq = []
#for x in dow30_df.: # company loop
for index, row in dow30_df.iterrows():
    x = row['CIK']
    i = 0
    currentDirectory = os.getcwd()
    file_dir = currentDirectory + '/data/download_2/'
    file_dir = file_dir + 'sec-edgar-filings/' + str(x).zfill(10) + '/10-Q/'
    for name in os.listdir(file_dir): 
        if not name.startswith('.') and os.path.isfile(os.path.join(file_dir,name)+'/full-submission.txt'):
            
            if i >=18: # only get 2 10K files from each company
                break
            print(os.path.join(file_dir,name)+'/full-submission.txt')
            print(row['Symbol'])
            item7 = parse_10q(row['Symbol'], os.path.join(file_dir,name)+'/full-submission.txt')
            print(len(item7))
            bag_lenq.append(len(item7))
            contentq.append(item7)
            cikq.append(x)
            companyq.append(row['Symbol'])
            file_nameq.append(name)
            doc_typeq.append('10-Q')
            file_d = read_file_date(os.path.join(file_dir,name)+'/full-submission.txt')
            file_dateq.append(file_d)
            conformed_period_d = read_conformed_period_date(os.path.join(file_dir,name)+'/full-submission.txt')
            conformed_periodq.append(conformed_period_d)
            i = i+1
            #print(i)

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-16-007105/full-submission.txt
MMM
7: 3729589
7a: 50415
7a: 5083902
7a: 5084684
start    3729666
end      5084672
delta    1355006
Name: 1, dtype: int64
6828
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-18-005773/full-submission.txt
MMM
no item 2
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-17-007801/full-submission.txt
MMM
no item 2
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-18-004248/full-submission.txt
MMM
no item 2
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-16-008937/full-submission.txt
MMM
no item 2
0
/Users/wailunchung/Documents/GitHu

7: 24826
7: 1546210
7a: 25812
7a: 2633334
start    1546284
end      2633322
delta    1087038
Name: 1, dtype: int64
7694
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000004962/10-Q/0000004962-18-000095/full-submission.txt
AXP
7: 22353
7: 1714363
7a: 23197
7a: 3257282
start    1714437
end      3257270
delta    1542833
Name: 1, dtype: int64
7266
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000004962/10-Q/0000004962-19-000031/full-submission.txt
AXP
7: 22515
7: 1420046
7a: 23410
7a: 2431108
start    1420120
end      2431096
delta    1010976
Name: 1, dtype: int64
6610
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000004962/10-Q/0000004962-18-000060/full-submission.txt
AXP
7: 21156
7: 1378619
7a: 22008
7a: 2338968
start    1378693
end      2338956
delta     960263
Name: 1, dtype: int64
7117
/Users/wailunchung/Documents/GitHub/DS6120_

7: 26791
7: 998478
7: 1295325
7a: 27871
7a: 1285757
7a: 1286507
start     998552
end      1286495
delta     287943
Name: 2, dtype: int64
3698
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320193/10-Q/0000320193-22-000007/full-submission.txt
AAPL
7: 128734
7: 909375
7a: 130155
7a: 1014761
7a: 1015231
start     909449
end      1015219
delta     105770
Name: 2, dtype: int64
1494
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320193/10-Q/0000320193-19-000066/full-submission.txt
AAPL
7: 260703
7: 1827925
7: 2095569
7a: 261783
7a: 2085661
7a: 2086409
start    1827999
end      2086397
delta     258398
Name: 2, dtype: int64
3119
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320193/10-Q/0000320193-20-000052/full-submission.txt
AAPL
7: 250636
7: 1736315
7a: 251716
7a: 1988209
7a: 1988957
start    1736389
end      1988945
delta     

3050
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000018230/10-Q/0000018230-21-000260/full-submission.txt
CAT
7: 582826
7: 4028828
7: 4029169
7: 4611813
7a: 584178
7a: 5252616
start    4611887
end      5252604
delta     640717
Name: 1, dtype: int64
2213
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000018230/10-Q/0000018230-16-000630/full-submission.txt
CAT
7: 18400
7: 3739469
7: 4515352
7a: 19523
7a: 5506961
start    4515426
end      5506949
delta     991523
Name: 1, dtype: int64
4101
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000018230/10-Q/0000018230-20-000297/full-submission.txt
CAT
7: 665407
7: 4216355
7: 4216696
7: 4802117
7a: 666759
7a: 5736355
start    4802191
end      5736343
delta     934152
Name: 1, dtype: int64
3473
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0

7: 184630
7: 1389799
7a: 185692
7a: 1660712
start    1389873
end      1660700
delta     270827
Name: 1, dtype: int64
3221
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000093410/10-Q/0000093410-18-000016/full-submission.txt
CVX
7: 26642
7: 804147
7a: 27557
7a: 1075331
start     804221
end      1075319
delta     271098
Name: 1, dtype: int64
3225
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000093410/10-Q/0000093410-18-000029/full-submission.txt
CVX
7: 26649
7: 975822
7a: 27564
7a: 1427674
start     975896
end      1427662
delta     451766
Name: 1, dtype: int64
3878
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000093410/10-Q/0000093410-18-000035/full-submission.txt
CVX
7: 27618
7: 980137
7a: 28680
7a: 1431371
start     980211
end      1431359
delta     451148
Name: 1, dtype: int64
4023
/Users/wailunchung/Documents/GitHub/DS6120_C

...

7: 28412
7: 1257022
7a: 30033
7a: 2111628
start    1257096
end      2111616
delta     854520
Name: 1, dtype: int64
9142
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000086312/10-Q/0000086312-21-000034/full-submission.txt
TRV
7: 293661
7: 2530615
7a: 295475
7a: 3313872
start    2530691
end      3313860
delta     783169
Name: 1, dtype: int64
11286
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000086312/10-Q/0001104659-17-045895/full-submission.txt
TRV
7: 2469525
7a: 3427921
start    2469601
end      3427909
delta     958308
Name: 0, dtype: int64
10926
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000086312/10-Q/0001104659-16-133593/full-submission.txt
TRV
7: 2365486
7a: 3265412
start    2365562
end      3265400
delta     899838
Name: 0, dtype: int64
10790
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/downloa

7: 46474
7: 1381048
7a: 47461
7a: 1853184
start    1381124
end      1853172
delta     472048
Name: 1, dtype: int64
2296
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000731766/10-Q/0000731766-17-000028/full-submission.txt
UNH
7: 46452
7: 1371262
7a: 47439
7a: 1844940
start    1371338
end      1844928
delta     473590
Name: 1, dtype: int64
2258
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000731766/10-Q/0000731766-21-000021/full-submission.txt
UNH
7: 260283
7: 1597602
7a: 261769
7a: 1833589
start    1597678
end      1833577
delta     235899
Name: 1, dtype: int64
2589
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000731766/10-Q/0000731766-21-000051/full-submission.txt
UNH
7: 258557
7: 1662815
7a: 260043
7a: 1995705
start    1662891
end      1995693
delta     332802
Name: 1, dtype: int64
2615
/Users/wailunchung/Documents/GitHub/DS6

7: 24909
7: 1395120
7: 1810310
7a: 25758
7a: 1857613
start    1810384
end      1857601
delta      47217
Name: 1, dtype: int64
576
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001403161/10-Q/0001403161-20-000012/full-submission.txt
V
7: 158552
7: 1241619
7a: 159401
7a: 1535462
start    1241693
end      1535450
delta     293757
Name: 1, dtype: int64
2006
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001403161/10-Q/0001403161-20-000048/full-submission.txt
V
7: 200350
7: 2176103
7a: 201959
7a: 2782077
start    2176177
end      2782065
delta     605888
Name: 1, dtype: int64
2812
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001403161/10-Q/0001403161-17-000028/full-submission.txt
V
7: 23837
7: 1011223
7: 1515358
7a: 24686
7a: 1592069
start    1515432
end      1592057
delta      76625
Name: 1, dtype: int64
753
/Users/wailunchung/Docume

7: 28298
7: 708458
7: 717589
7: 719084
7: 1124521
7: 1170410
7: 1231667
7a: 29249
7a: 1122005
start     719158
end      1121993
delta     402835
Name: 1, dtype: int64
3428
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000104169/10-Q/0000104169-18-000054/full-submission.txt
WMT
7: 30411
7: 676208
7: 690534
7: 691784
7: 1004290
7: 1052380
7: 1116076
7a: 31362
7a: 1000260
start     691858
end      1000248
delta     308390
Name: 1, dtype: int64
3226
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000104169/10-Q/0000104169-17-000057/full-submission.txt
WMT
7: 30428
7: 763425
7: 772040
7: 773513
7: 1176834
7: 1237350
7: 1300062
7a: 31379
7a: 1173075
start     773587
end      1173063
delta     399476
Name: 1, dtype: int64
3470
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000104169/10-Q/0000104169-21-000058/full-submission.txt
WMT
7: 1771

7: 1826783
7: 2425586
7a: 2425324
start    1826857
end      2425312
delta     598455
Name: 0, dtype: int64
6200
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001744489/10-Q/0001744489-22-000059/full-submission.txt
DIS
7: 1712814
7: 2445536
7a: 2445274
start    1712888
end      2445262
delta     732374
Name: 0, dtype: int64
7051
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001744489/10-Q/0001744489-21-000181/full-submission.txt
DIS
7: 2563837
7: 3579110
7a: 3578848
start    2563911
end      3578836
delta    1014925
Name: 0, dtype: int64
9270


### combining 10k and 10q

In [94]:
company = []
file_name = []
doc_type = []
file_date = []
content = []
cik = []
conformed_period = []
bag_len = []
company = companyk + companyq
file_name = file_namek+ file_nameq
doc_type = doc_typek + doc_typeq
file_date = file_datek + file_dateq
content = contentk + contentq
cik = cikk + cikq
conformed_period = conformed_periodk + conformed_periodq
bag_len = bag_lenk + bag_lenq

In [95]:
all_df = pd.DataFrame(
    {'company': company,
     'cik': cik,
     'file_name': file_name,
     'doc_type': doc_type,
     'file_date': file_date,
     'conformed_period': conformed_period,
     'bag_len': bag_len,
     'content': content
    })

In [96]:
all_df.groupby(['company', 'doc_type']).size()

company  doc_type
AAPL     10-K         6
         10-Q        18
AMGN     10-K         6
         10-Q        18
AXP      10-K         6
         10-Q        18
BA       10-K         6
         10-Q        18
CAT      10-K         6
         10-Q        18
CRM      10-K         6
         10-Q        18
CSCO     10-K         6
         10-Q        18
CVX      10-K         6
         10-Q        18
DIS      10-K         3
         10-Q         9
DOW      10-K         3
         10-Q         9
GS       10-K         6
         10-Q        18
HD       10-K         6
         10-Q        18
HON      10-K         6
         10-Q        18
IBM      10-K         6
         10-Q        18
INTC     10-K         6
         10-Q        18
JNJ      10-K         6
         10-Q        18
JPM      10-K         6
         10-Q        18
KO       10-K         6
         10-Q        18
MCD      10-K         6
         10-Q        18
MMM      10-K         6
         10-Q        18
MRK      10-K         

## check how many pass minimum word count 


In [97]:
all_df

,company,cik,file_name,doc_type,file_date,conformed_period,bag_len,content
0,MMM,66740,0001558370-17-000479,10-K,20170209,20161231,10074,"[designed, provide, reader, financial, stateme..."
1,MMM,66740,0001558370-19-000470,10-K,20190207,20181231,11067,"[designed, provide, reader, financial, stateme..."
2,MMM,66740,0001558370-21-000737,10-K,20210204,20201231,10765,"[sections, company, annual, report, form, year..."
3,MMM,66740,0001558370-20-000581,10-K,20200206,20191231,10676,"[designed, provide, reader, financial, stateme..."
4,MMM,66740,0000066740-22-000010,10-K,20220209,20211231,7403,"[sections, company, annual, report, form, year..."
...,...,...,...,...,...,...,...,...
691,DIS,1744489,0001744489-21-000108,10-Q,20210513,20210403,9165,"[organization, informationmanagement, discussi..."
692,DIS,1744489,0001744489-19-000167,10-Q,20190806,20190629,7905,"[organization, informationmanagement, discussi..."
693,DIS,1744489,0001744489-21-000047,10-Q,20210211,20210102,6200,"[organization, informationmanagement, discussi..."
694,DIS,1744489,0001744489-22-000059,10-Q,20220209,20220101,7051,"[quarterly, report, form, contains, forward, l..."


In [98]:
all_df[all_df.bag_len > 500]

,company,cik,file_name,doc_type,file_date,conformed_period,bag_len,content
0,MMM,66740,0001558370-17-000479,10-K,20170209,20161231,10074,"[designed, provide, reader, financial, stateme..."
1,MMM,66740,0001558370-19-000470,10-K,20190207,20181231,11067,"[designed, provide, reader, financial, stateme..."
2,MMM,66740,0001558370-21-000737,10-K,20210204,20201231,10765,"[sections, company, annual, report, form, year..."
3,MMM,66740,0001558370-20-000581,10-K,20200206,20191231,10676,"[designed, provide, reader, financial, stateme..."
4,MMM,66740,0000066740-22-000010,10-K,20220209,20211231,7403,"[sections, company, annual, report, form, year..."
...,...,...,...,...,...,...,...,...
691,DIS,1744489,0001744489-21-000108,10-Q,20210513,20210403,9165,"[organization, informationmanagement, discussi..."
692,DIS,1744489,0001744489-19-000167,10-Q,20190806,20190629,7905,"[organization, informationmanagement, discussi..."
693,DIS,1744489,0001744489-21-000047,10-Q,20210211,20210102,6200,"[organization, informationmanagement, discussi..."
694,DIS,1744489,0001744489-22-000059,10-Q,20220209,20220101,7051,"[quarterly, report, form, contains, forward, l..."


In [99]:
df_500 = all_df[all_df.bag_len > 500]

## save to CSV

In [100]:
df_500.to_csv('data_10k10q.csv', sep='|',index=False)

In [101]:
## another way ##

In [23]:
import csv

with open('data_10k10q.csv', 'w') as f:
    writer = csv.writer(f,delimiter ='|') # pipe delimited
    writer.writerow(['company', 'file_name', 'doc_type','file_date', 'cik', 'conformed_period','bag_len','content'])
    writer.writerows(zip(company, file_name, doc_type, file_date, cik, conformed_period, bag_len, content))

In [74]:
################ BELOW are parsing test ##########################

## TEST 1 company 10-K

In [472]:
companyq = []
file_nameq = []
doc_typeq = []
file_dateq = []
contentq = []
cikq = []
conformed_periodq = []
bag_lenq = []
#dow30_df_test = 0001618921
dow30_df_test = dow30_df.loc[dow30_df['CIK'] == 1618921]
#for x in dow30_df.: # company loop
for index, row in dow30_df_test.iterrows():
    x = row['CIK']
    i = 0
    currentDirectory = os.getcwd()
    file_dir = currentDirectory + '/data/download_2/'
    file_dir = file_dir + 'sec-edgar-filings/' + str(x).zfill(10) + '/10-K/'
    for name in os.listdir(file_dir): 
        if not name.startswith('.') and os.path.isfile(os.path.join(file_dir,name)+'/full-submission.txt'):
            
            if i >=10: # only get 2 10K files from each company
                break
            print(os.path.join(file_dir,name)+'/full-submission.txt')
            print(row['Symbol'])
            item7 = parse_10k(row['Symbol'], os.path.join(file_dir,name)+'/full-submission.txt')
            print(len(item7))
            bag_lenq.append(len(item7))
            contentq.append(item7)
            cikq.append(x)
            companyq.append(row['Symbol'])
            file_nameq.append(name)
            doc_typeq.append('10-K')
            file_d = read_file_date(os.path.join(file_dir,name)+'/full-submission.txt')
            file_dateq.append(file_d)
            conformed_period_d = read_conformed_period_date(os.path.join(file_dir,name)+'/full-submission.txt')
            conformed_periodq.append(conformed_period_d)
            i = i+1
            #print(i)

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001618921-21-000085/full-submission.txt
WBA
7: 387115
7: 723526
7a: 388293
7a: 1328222
start     723602
end      1328210
delta     604608
Name: 1, dtype: int64
9303
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001628280-18-012472/full-submission.txt
WBA
7: 32800
7: 526262
7a: 33728
7a: 1228684
start     526338
end      1228672
delta     702334
Name: 1, dtype: int64
8180
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001618921-20-000082/full-submission.txt
WBA
7: 378561
7: 871567
7a: 380169
7a: 1657055
start     871643
end      1657043
delta     785400
Name: 1, dtype: int64
9683
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001618921-19-000069/full-submission.txt
WBA
7: 

## TEST 1 company 10-Q

In [159]:
companyq = []
file_nameq = []
doc_typeq = []
file_dateq = []
contentq = []
cikq = []
conformed_periodq = []
bag_lenq = []
#dow30_df_test = 0001618921
dow30_df_test = dow30_df.loc[dow30_df['CIK'] == 789019]
#for x in dow30_df.: # company loop
for index, row in dow30_df_test.iterrows():
    x = row['CIK']
    i = 0
    currentDirectory = os.getcwd()
    file_dir = currentDirectory + '/data/download_2/'
    file_dir = file_dir + 'sec-edgar-filings/' + str(x).zfill(10) + '/10-Q/'
    for name in os.listdir(file_dir): 
        if not name.startswith('.') and os.path.isfile(os.path.join(file_dir,name)+'/full-submission.txt'):
            
            if i >=10: # only get 2 10K files from each company
                break
            print(os.path.join(file_dir,name)+'/full-submission.txt')
            print(row['Symbol'])
            item7 = parse_10q(row['Symbol'], os.path.join(file_dir,name)+'/full-submission.txt')
            print(len(item7))
            bag_lenq.append(len(item7))
            contentq.append(item7)
            cikq.append(x)
            companyq.append(row['Symbol'])
            file_nameq.append(name)
            doc_typeq.append('10-K')
            file_d = read_file_date(os.path.join(file_dir,name)+'/full-submission.txt')
            file_dateq.append(file_d)
            conformed_period_d = read_conformed_period_date(os.path.join(file_dir,name)+'/full-submission.txt')
            conformed_periodq.append(conformed_period_d)
            i = i+1
            #print(i)

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-21-051992/full-submission.txt
MSFT
7: 245100
7: 3241404
7a: 247225
7a: 3659649
start    3241480
end      3659637
delta     418157
Name: 1, dtype: int64
4669
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-19-001392/full-submission.txt
MSFT


Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x127b5b400>
Traceback (most recent call last):
  File "/Users/wailunchung/.pyenv/versions/3.7.3/lib/python3.7/site-packages/nltk/data.py", line 1161, in __del__
    if not self.closed:
  File "/Users/wailunchung/.pyenv/versions/3.7.3/lib/python3.7/site-packages/nltk/data.py", line 1175, in closed
    return self.stream.closed
AttributeError: 'SeekableUnicodeStreamReader' object has no attribute 'stream'


7: 44310
7a: 46363
start    44386
end      46351
delta     1965
Name: 0, dtype: int64
6
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-20-002450/full-submission.txt
MSFT
7: 297508
7: 4130943
7a: 299633
7a: 4778313
start    4131019
end      4778301
delta     647282
Name: 1, dtype: int64
4536
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-19-012709/full-submission.txt
MSFT
7: 44126
7a: 46179
start    44202
end      46167
delta     1965
Name: 0, dtype: int64
6
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-21-020891/full-submission.txt
MSFT
7: 327232
7: 4301797
7a: 329357
7a: 4973811
start    4301873
end      4973799
delta     671926
Name: 1, dtype: int64
5762
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/00

# step by step testing

In [7]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000)

3000


In [431]:
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/CAT/10-K/0000018230-21-000063/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/GS/10-K/0001193125-21-102511/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/IBM/10-K/0001558370-20-001786/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/MRK/10-Q/0000310158-20-000009/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/HON/10-Q/0000773840-20-000038/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-19-012709/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000773840/10-K/0000930413-17-000391/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000019617/10-K/0000019617-20-000257/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000019617/10-K/0000019617-21-000236/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001618921-17-000069/full-submission.txt'


In [432]:

f = open(filepath)
raw_10k = f.read()

In [433]:
import re
# Regex to find <DOCUMENT> tags
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')
doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

In [434]:
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

In [435]:
document = {}

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K':
        print(doc_start)
        print(doc_end)
        #doc_10k = raw_10k[doc_start:doc_end].lower()
        document[doc_type] = raw_10k[doc_start:doc_end].lower()
        doc_10k = raw_10k[doc_start:doc_end].lower()
        break

986
3343971


In [436]:
len(document['10-K'])

3342985

In [437]:
doc_10k = BeautifulSoup(doc_10k, 'lxml')

In [438]:
doc_10k_str = str(doc_10k)

In [465]:

i7 = 's discussion and analysis of financial condition and results of operations'
#i7 = 's discussion and analysis of financial condition and results of operations'
#JPM
#i7 = 'introduction'
#i7 = 'introduction </span></div>'
#MSFT 2016 
#i7 = 's discussion and analysis of financial condition and results of operations </b>'
# V 2019+
#i7 = 's discussion and analysis of financial condition and results of operations</span></div>'
#i7a = 'quantitative and qualitative disclosures about market risk</span></div>'
# V 2019<
#i7 = 'd;">management’s discussion and analysis of financial condition and results of operations</font></div></td>'
#i7a = 'quantitative and qualitative disclosures about market risk</font></div>'
# WBA
i7 = 's discussion and analysis of financial condition and results of operations</'
sec7_find = re.finditer(i7, doc_10k_str)
i7a = 'quantitative and qualitative disclosures about market risk'
#i7a = 'quantitative and qualitative disclosure about market risk'
i7a = 'quantitative and qualitative disclosures about market risk'
# WBA
i7 = 's discussion and analysis of financial condition and results of operations</'
i7a = 'quantitative and qualitative disclosure about market risk</'


## JPM 
#i7a = 's report on internal control over financial reporting'
#i7a = 's report on internal control over financial reporting</span></div>'
# MCD 
#i7a = 'other key information'
#i7a = 'item\xa04. controls and procedures'

sec7a_find = re.finditer(i7a, doc_10k_str)

#for x in sec7a_find: 
#    print(x)
sec7_df = None
sec7a_df = None
if sec7_df is not None:
    del sec7_df
if sec7a_df is not None:
    del sec7a_df

    # moving keyword range to dataframe
sec7_start_range = []
sec7_end_range = []
for x in sec7_find: 
    #print(x.start())
    sec7_start_range.append(x.start())
    sec7_end_range.append(x.end())
    sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
sec7a_start_range = []
sec7a_end_range = []    
for x in sec7a_find: 
    sec7a_start_range.append(x.start())
    sec7a_end_range.append(x.end())
    sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})


In [466]:
#del sec7a_df

In [467]:
for index, row in sec7_df.iterrows():
    print('7' , str(row['start']) , str(row['end']))
for index, row in sec7a_df.iterrows():
    print('7a', str(row['start']), str(row['end']))


7 32944 33020
7 518561 518637
7a 1209636 1209695


In [468]:
for index, row in sec7_df.iterrows(): 
    print(doc_10k_str[row['start']-100:row['end']+100])
    print('------')
print('####')
for index, row in sec7a_df.iterrows(): 
    print(doc_10k_str[row['start']-100:row['end']+100])
    print('------')


="font-family:inherit;font-size:10pt;"><font style="font-family:inherit;font-size:10pt;">management’s discussion and analysis of financial condition and results of operations</font></a></div></td><td style="vertical-align:top;background-color:#cceeff;padding-left:2px;padding-
------
t style="font-family:inherit;font-size:10pt;font-weight:bold;text-decoration:underline;">management’s discussion and analysis of financial condition and results of operations</font></div><div style="line-height:120%;text-align:left;font-size:10pt;"><font style="font-family:in
------
####
</font><font style="font-family:inherit;font-size:10pt;font-weight:bold;text-decoration:underline;">quantitative and qualitative disclosure about market risk</font></div><div style="line-height:120%;text-align:left;font-size:10pt;"><font style="font-family:in
------


In [242]:
item_7_content = doc_10k_str[507977:548190]
sss = preprocess(item_7_content)
bagOfWordsA = sss.split(' ')

In [243]:
len(bagOfWordsA)

1670

In [182]:
doc_10k_str[2720768:2721042]

'cies are considered critical as disclosed in the critical accounting policies section of management’s discussion and analysis of financial condition and results of operations included in merck’s form 10-k because of the potential for a significant impact on the financial st'

In [50]:
i7 = "discussion and analysis of financial condition and results of operations"
#i7 = "financial condition and results of operations"
sec7_find = re.finditer(i7, doc_10k)

In [51]:
for x in sec7_find: 
    print(x.start())

30319
518530
716604


In [145]:
doc_10k[1087900:1132112]

'uding successfully integrating acquired businesses, could have an adverse effect on our business, financial condition and results of operations.&#160; Furthermore, we make strategic divestitures from time to time. In the case of divestitures, we may agree to indemnify acquiring parties for certain liabilities arising from our former businesses. These divestitures may also result in continued financial involvement in the divested businesses following the transaction, including through guarantees or other financial arrangements.&#160; Lower performance by those divested businesses could affect our future financial results.</span></div><div><span><br/></span></div><div><span style="color:#000000;font-family:\'Times New Roman\',sans-serif;font-size:10pt;font-weight:700;line-height:120%">Union disputes or other labor matters could adversely affect our operations and financial results.</span></div><div><span style="color:#000000;font-family:\'Times New Roman\',sans-serif;font-size:10pt;font

In [462]:
#i7 = 'management&#8217;s discussion and analysis of financial condition and results of operations'
i7 = "s discussion and analysis of financial condition and results of operations"
sec7_find = re.finditer(i7, doc_10k_str)
i7a = 'quantitative and qualitative disclosures about market risk' 
sec7a_find = re.finditer(i7a, doc_10k_str)
    
# moving keyword range to dataframe
sec7_start_range = []
sec7_end_range = []
for x in sec7_find: 
    #print(x.start())
    #print(x.end())
    sec7_start_range.append(x.start())
    sec7_end_range.append(x.end())
    sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
sec7a_start_range = []
sec7a_end_range = []    
for x in sec7a_find: 
    #print(x.start())
    #print(x.end())
    sec7a_start_range.append(x.start())
    sec7a_end_range.append(x.end())
    sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})
    
#qc check
for index, row in sec7_df.iterrows():
    print(row['start'], row['end'])
    print('item 7')
for index, row in sec7a_df.iterrows():
    print(row['start'], row['end'])
7 125654 125728
7 591991 592065
7 593414 593488
7a 127199 127257
7a 632141 632199

125654 125728
item 7
591991 592065
item 7
593414 593488
item 7
127199 127257
632141 632199


In [475]:
    
# get possible ranges 
possible_start_range = []
possible_end_range = []
for index, row7a in sec7a_df.iterrows():
    filtered = []
    for i, row7 in sec7_df.iterrows():
        if row7['end'] < row7a['start']:
            filtered.append(row7['end'])
            
    if not filtered:
        continue # nothing before item 7a
    #print(filtered)
    max_y = max(filtered)
    possible_start_range.append(max_y)
    possible_end_range.append(row7a['start']-12)
    
    # qc 
    #for x in possible_start_range:
    #    print(x)
    
    # get the largest possible range
possible_df = pd.DataFrame({'start': possible_start_range,'end': possible_end_range})
possible_df['delta'] = possible_df['end'] - possible_df['start']
    
possible_df = possible_df.loc[possible_df['delta'].idxmax()]


In [476]:
possible_df

start    593488
end      632129
delta     38641
Name: 1, dtype: int64

In [477]:
item_7_content = doc_10k_str[possible_df['start']:possible_df['end']]
sss = preprocess(item_7_content)
bagOfWordsA = sss.split(' ')

In [478]:
len(bagOfWordsA)

1802

In [472]:
bagOfWordsA[1700:1803]


['expense',
 'attribution',
 'three',
 'months',
 'six',
 'months',
 'ended',
 'june',
 'june',
 'subscription',
 'agreements',
 'subscription',
 'agreements',
 'involve',
 'physical',
 'settlement',
 'fixed',
 'number',
 'qualifies',
 'equity',
 'classification',
 'accounting',
 'standards',
 'codification',
 'asc',
 'derivatives',
 'hedging',
 'therefore',
 'periodically',
 'remeasured',
 'fair',
 'value',
 'backstop',
 'agreement',
 'backstop',
 'agreement',
 'involves',
 'conditional',
 'obligation',
 'company',
 'must',
 'settle',
 'issuing',
 'variable',
 'number',
 'shares',
 'monetary',
 'value',
 'predominantly',
 'based',
 'variations',
 'something',
 'fair',
 'value',
 'company',
 'shares',
 'initially',
 'subsequently',
 'measured',
 'fair',
 'value',
 'accounting',
 'standards',
 'codification',
 'asc',
 'distinguishing',
 'liabilities',
 'equity',
 'recent',
 'accounting',
 'pronouncements',
 'management',
 'believe',
 'recently',
 'issued',
 'yet',
 'effective',
 'accoun